(quick-start)=

# A Quick Start Tutorial

This tutorial provides a quick start guide to using ``ELISA`` to fit a spectral model to X-ray spectral data. The tutorial is divided into three sections:

1. Load Data and Define Spectral Model
2. Bayesian Fit
3. Maximum Likelihood Fit

The data used in this tutorial is from the Insight-HXMT observation of the Crab Nebula. The spectra are obtained by the Low Energy (LE), the Medium Energy (ME), and the High Energy (HE) telescopes. The spectral model used in this tutorial is a power-law model modified by a photoelectric absorption.

The tutorial demonstrates how to fit the spectral model to the data using both Bayesian and Maximum Likelihood methods.

```{tip}
**CPU Core Configuration:** By default, ELISA uses 4 available CPU cores for parallel computation. To customize the number of cores, add the following code at the beginning of your script:

​```python
import elisa
elisa.set_cpu_cores(4)  # Set to your desired number of cores
​```

Note that this setting only affects CPU-based computations. Users running on non-CPU devices (e.g., GPU) can ignore this configuration.
```

## 1. Load Data and Define Spectral Model

In [ ]:
from elisa import BayesFit, Data, MaxLikeFit
from elisa.models import PhAbs, PowerLaw

LE = Data(
    erange=[1.5, 10],
    specfile='data/P011160506306_LE_TOTAL.fits',
    backfile='data/P011160506306_LE_BKG.fits',
    respfile='data/P011160506306_LE_RSP.fits',
    group='opt',
)

ME = Data(
    erange=[10, 35],
    specfile='data/P011160506306_ME_TOTAL.fits',
    backfile='data/P011160506306_ME_BKG.fits',
    respfile='data/P011160506306_ME_RSP.fits',
    group='opt',
)

HE = Data(
    erange=[28, 250],
    specfile='data/P011160506306_HE_TOTAL.fits',
    backfile='data/P011160506306_HE_BKG.fits',
    respfile='data/P011160506306_HE_RSP.fits',
    group='opt',
)

data = [LE, ME, HE]

model = PhAbs() * PowerLaw()

## 2. Bayesian Fit

In [ ]:
fit = BayesFit(data, model)
fit

### Run the No-U-Turn Sampler

In [ ]:
posterior = fit.nuts(chains=4, progress=False)
posterior

### Plot the Trace of Sampler

In [ ]:
fig = posterior.plot.plot_trace()

### Plot of Spectral Fit and Residuals

In [ ]:
fig = posterior.plot()

### Goodness of Fit: Quantile-Quantile Plot

In [ ]:
fig = posterior.plot.plot_qq(detrend=False)

### Goodness of Fit: Probability Integral Transform ECDF Plot

In [ ]:
fig = posterior.plot.plot_pit(detrend=False)

### Plot Marginal Posterior Distribution

In [ ]:
fig = posterior.plot.plot_corner()

### Calculate Credible Intervals of Parameters

In [ ]:
ci = posterior.ci()
ci

In [ ]:
ci.median

In [ ]:
ci.errors

## 3. Maximum Likelihood Fit

In [ ]:
model.PhAbs.nH.default = 0.35
fit2 = MaxLikeFit([LE, ME, HE], model)
fit2

### Use Levenberg-Marquardt algorithm to find the MLE

In [ ]:
mle = fit2.mle(method='lm')
mle

### Plot of Spectral Fit and Residuals

In [ ]:
fig = mle.plot()

### Goodness of Fit: Quantile-Quantile Plot

In [ ]:
fig = mle.plot.plot_qq(detrend=False)

### Goodness of Fit: Probability Integral Transform ECDF Plot

In [ ]:
fig = mle.plot.plot_pit(detrend=False)

### Calculate Confidence Intervals of Parameters

In [ ]:
ci = mle.ci()
ci

In [ ]:
ci.mle

In [ ]:
ci.errors